<a href="https://colab.research.google.com/github/Apsod/DD2360HT22/blob/main/hw_2/ex_4/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
!tar -xf rodinia_3.1.tar.bz2

--2022-11-28 16:50:36--  http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Resolving www.cs.virginia.edu (www.cs.virginia.edu)... 128.143.67.11
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2 [following]
--2022-11-28 16:50:37--  https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360636246 (344M) [application/x-tar]
Saving to: ‘rodinia_3.1.tar.bz2’

rodinia_3.1.tar.bz2 100%[===================>] 343.93M  12.8MB/s    in 28s     

2022-11-28 16:51:07 (12.1 MB/s) - ‘rodinia_3.1.tar.bz2’ saved [360636246/360636246]



In [ ]:
TXT= """
CUDA_DIR = /usr/local/cuda
CUDA_LIB_DIR = ${CUDA_DIR}/lib64
SDK_DIR = ${CUDA_DIR}/samples/
""".strip()

with open('rodinia_3.1/common/make.config', 'wt') as handle:
  handle.write(
    TXT      
  )

In [ ]:
from pathlib import Path 
import subprocess
import itertools

ROOT = Path('.').absolute()

implementations = ['cuda', 'openmp']
problems = ['backprop', 'nw', 'nn']
for impl, problem in itertools.product(implementations, problems):
  path = ROOT / 'rodinia_3.1' / impl / problem
  print(f'\n\n================\n\nmaking {impl} {problem}\n{path}')
  p = subprocess.run(f"cd {path}; make", shell=True, capture_output=True)
  print(p.stdout.decode('utf-8'))
  print(p.stderr.decode('utf-8'))




making cuda backprop
/content/rodinia_3.1/cuda/backprop
make: 'backprop' is up to date.





making cuda nw
/content/rodinia_3.1/cuda/nw
/usr/local/cuda/bin/nvcc  needle.cu -o needle -I/usr/local/cuda/include -L/usr/local/cuda/lib64 





making cuda nn
/content/rodinia_3.1/cuda/nn
make: Nothing to be done for 'all'.





making openmp backprop
/content/rodinia_3.1/openmp/backprop
make: 'backprop' is up to date.





making openmp nw
/content/rodinia_3.1/openmp/nw
g++ -g -O3 -fopenmp needle.cpp -o needle 
icc -g -O3 -fopenmp -offload-option,mic,compiler,"-no-opt-prefetch" -DOMP_OFFLOAD needle.cpp -o needle_offload
Makefile:13: recipe for target 'needle_offload' failed

/bin/sh: 1: icc: not found
make: *** [needle_offload] Error 127




making openmp nn
/content/rodinia_3.1/openmp/nn
make: Nothing to be done for 'all'.




In [85]:
# runmap for varying the sizes: making the neural net in backprop larger,
# and the sequences in Needleman-Wunsch longer. 

runmap = {
    ('cuda', 'backprop'): './backprop 524288',
    ('openmp', 'backprop'): './backprop 524288',
    ('cuda', 'nw'): './needle 8192 10',
    ('openmp', 'nw'): './needle 8192 10 2',
    ('cuda', 'nn'): './run',
    ('openmp', 'nn'): './run',
}

In [86]:
from time import perf_counter

for problem, impl in itertools.product(problems, implementations):
  path = ROOT / 'rodinia_3.1' / impl / problem
  print(f'\n=======\nrunning {impl} {problem} @ {path}')
  start = perf_counter()
  p = subprocess.run(f"cd {path}; {runmap[(impl, problem)]}", shell=True, capture_output=True)
  stop = perf_counter()
  print(f'elapsed time: {stop - start} seconds')
  print('STDOUT')
  print(p.stdout.decode('utf-8'))
  print('STDERR')
  print(p.stderr.decode('utf-8'))




running cuda backprop @ /content/rodinia_3.1/cuda/backprop
elapsed time: 0.6974927889987157 seconds
STDOUT
Random number generator seed: 7
Input layer size : 524288
Starting training kernel
Performing GPU computation
Training done

STDERR


running openmp backprop @ /content/rodinia_3.1/openmp/backprop
elapsed time: 0.5912364480009273 seconds
STDOUT
Random number generator seed: 7
Input layer size : 524288
Starting training kernel
Performing CPU computation
Training done

STDERR


running cuda nw @ /content/rodinia_3.1/cuda/nw
elapsed time: 1.1790974909999932 seconds
STDOUT
WG size of kernel = 16 
Start Needleman-Wunsch
Processing top-left matrix
Processing bottom-right matrix

STDERR


running openmp nw @ /content/rodinia_3.1/openmp/nw
elapsed time: 0.7146968919987557 seconds
STDOUT
Start Needleman-Wunsch
Num of threads: 2
Processing top-left matrix
Processing bottom-right matrix
Total time: 0.383 seconds

STDERR


running cuda nn @ /content/rodinia_3.1/cuda/nn
elapsed time: 0.173586

In [ ]:
!ls /content/rodinia_3.1/cuda/backprop

backprop		 backprop_cuda.o  facetrain.o  Makefile_nvidia
backprop.c		 backprop.h	  imagenet.c   run
backprop_cuda.cu	 backprop.o	  imagenet.o
backprop_cuda_kernel.cu  facetrain.c	  Makefile


make: *** No targets specified and no makefile found.  Stop.
